# Obtaining data

We use blaseball_mike as an wrapper to query the Datablase API.
First we obtain all stats for all players, and write the raw data into JSONs.

In [10]:
from blaseball_mike import reference as r
from blaseball_mike.models import Team
import json
import pandas as pd
import qgrid

In [4]:
p_stats = r.get_stats()

In [12]:
with open("player_stats.json", "w") as f:
    json.dump(p_stats, f)

# EDA

Now we want to split up the raw data between hitters and pitchers and look at the stats in a spreadsheet format.
## Hitting data

In [180]:
hitting_df = pd.DataFrame([[i["player"]["fullName"]]+[i["player"]["id"]]+[i["team"]["team_abbreviation"]]+[i["team"]["team_id"]]+list(i["stat"].values()) for i in p_stats[0]["splits"]], 
                         columns = ["name","id", "team_abb", "team_id"]+list(p_stats[0]["splits"][0]["stat"].keys()))
import qgrid
q_widget = qgrid.show_grid(hitting_df, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":50})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

### Note
Some players may have more than one row (this data describes one season). This is because a player may be Feedbacked
to another team in the middle of the season. Note that team names are currently bugged when this happens, so the 
team before and after will have the same name.

Note that "NaN" is an actual player and not an error.

"Duplicate" hitters:

In [46]:
g1 = hitting_df[["name", "team_abb"]].groupby("name").count()
g1.loc[g1["team_abb"] > 1]
#hitting_df.loc[hitting_df["name"] == "NaN"]

,team_abb
name,
Bates Bentley,2
Fletcher Yamamoto,2
NaN,3


## Pitching data

In [178]:
with open("player_stats.json", "r") as f:
    p_stats = json.load(f)
pitching_df = pd.DataFrame([[i["player"]["fullName"]]+[i["player"]["id"]]+[i["team"]["team_abbreviation"]]+[i["team"]["team_id"]]+list(i["stat"].values()) for i in p_stats[1]["splits"]], 
                         columns = ["name","id", "team_abb", "team_id"]+list(p_stats[1]["splits"][0]["stat"].keys()))

q_widget = qgrid.show_grid(pitching_df, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":50})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

"Duplicate" pitchers:

In [242]:
g2 = pitching_df[["name", "team_abb"]].groupby("name").count()
g2.loc[g2["team_abb"] > 1]

,team_abb
name,
Agan Harrison,2
Edric Tosser,2
James Mora,2
Jaylen Hotdogfingers,3
Yeong-Ho Garcia,2


In [49]:
hitting_df.to_csv("hitting.csv", index=False)
pitching_df.to_csv("pitching.csv", index=False)

## Relative Performance

It's useful to have a sense of what "good" looks like in a stat, so we look at some summary statistics that describe the distributions of different stats, and the quartiles of performance so we get an idea of what is above/below/exactly average.

Note that these stats ignore any player who has an N/A in their column, which usually means that they do not have enough data to properly compute a stat. There still may be some players who have played a low number of games and still end up being used in this frame though.

### Relative Hitting Performance

In [176]:
stats_frame = hitting_df.drop(columns=["id", "name", "team_abb", "team_id"], axis=1).dropna()
sum_stats = [stats_frame[col].describe().reset_index(drop=True) for col in stats_frame.columns]
h_summary = pd.DataFrame(sum_stats)
h_summary.columns = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]
q_widget = qgrid.show_grid(h_summary.round(2), show_toolbar=True)
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Relative Pitching Performance

In [175]:
stats_frame = pitching_df.drop(columns=["id", "name", "team_abb", "team_id"], axis=1).dropna()
sum_stats = [stats_frame[col].describe().reset_index(drop=True) for col in stats_frame.columns]
p_summary = pd.DataFrame(sum_stats)
p_summary.columns = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]
q_widget = qgrid.show_grid(p_summary.round(2), show_toolbar=True)
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Also do one with the player's names and equivalent percentile for each stat ?

In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs



# Team averages

Now we want to look at average stats for teams, to see who is performing the best overall.

## Getting averages

We just split the player data by team and write them to separate JSONs.

In [123]:
data_folder = "DATA/"
id_to_name = hitting_df[["team_id", "team_abb"]].drop_duplicates().to_dict("records")
for team in id_to_name:
    with open(data_folder+team["team_abb"]+".json", "w") as f:
        json.dump(r.get_stats(team_id=team["team_id"]), f)
        

Then we turn the JSONs into CSVs and calculate averages for all players in the sheet, to be stored as a row in the sheet under ["name"] `team_avg`.

# the file reading/writing might not work; I didn't test it

In [ ]:
csv_folder = data_folder+"CSV/"
for team in id_to_name:
    with open(data_folder+team["team_abb"]+".json", "r") as f:
        js = json.load(f)
        frame = pd.DataFrame([[i["player"]["fullName"]]+list(i["stat"].values()) for i in js[0]["splits"]], columns=["name"]+list(js[0]["splits"][0]["stat"].keys()))
        avgs = frame.mean(axis=0).round(3)
        avgs["name"] = "team_avg"
        frame = frame.append(avgs, ignore_index=True)
        frame.set_index(keys="name", inplace=True)
        frame.to_csv(csv_folder+team["team_abb"]+"_hitting.csv")
        
        frame = pd.DataFrame([[i["player"]["fullName"]]+list(i["stat"].values()) for i in js[1]["splits"]], columns=["name"]+list(js[1]["splits"][0]["stat"].keys()))
        avgs = frame.mean(axis=0).round(3)
        avgs["name"] = "team_avg"
        frame = frame.append(avgs, ignore_index=True)
        frame.set_index(keys="name", inplace=True)
        frame.to_csv(csv_folder+team["team_abb"]+"_pitching.csv")


Use these functions to get the tables with the averages; the team_avg data is not stored in the file but is calculated by these functions.

In [157]:
def get_team_hitting_avg_df():
    avgs = []
    for team in id_to_name:
        df = pd.read_csv(csv_folder+team["team_abb"]+"_hitting.csv", index_col="name")
        avg = df.iloc[-1]
        avgs.append(avg)
    avg_frame = pd.DataFrame(avgs, index = [team["team_abb"] for team in id_to_name])
    return avg_frame

def get_team_pitching_avg_df():
    avgs = []
    for team in id_to_name:
        df = pd.read_csv(csv_folder+team["team_abb"]+"_pitching.csv", index_col="name")
        avg = df.iloc[-1]
        avgs.append(avg)
    avg_frame = pd.DataFrame(avgs, index = [team["team_abb"] for team in id_to_name])
    return avg_frame

In [169]:
avg_hitting = get_team_hitting_avg_df()
q_widget = qgrid.show_grid(avg_hitting, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":70})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [182]:
avg_pitching = get_team_pitching_avg_df()
q_widget = qgrid.show_grid(avg_pitching, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":70})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

# It occurs to me that some of the stats are averages of averages, which doesn't work (unless corrected for by the number of games/appearances)

# Ranking
Now let's try ranking the hitters by several stats at once, using a rank fusion algorithm to calculate an overall ranking.

In [236]:
def ranker(df, labels, asc_list=None, top=5):
    if asc_list == None:
        asc_list = len(labels)*[True] 
    
    rankings = [df[label].rank(ascending=asc) for asc, label in zip(asc_list, labels)]
    frame = pd.DataFrame(rankings)
    sum_ = frame.sum(axis=0).round(3)
    try:
        sum_.index = df["name"]
    except KeyError:
        pass
    return(sum_.sort_values(ascending=False)[:top])

## Ranking Hitters

In [229]:
hitting_ranks = ranker(hitting_df, labels=["on_base_slugging", "total_bases", "batting_average", "runs_batted_in"], top=10)
hitting_ranks

name
Dudley Mueller          902.0
Aldon Cashmoney         897.0
Jaxon Buckley           888.0
Valentine Games         887.5
Nagomi Mcdaniel         881.0
Jode Preston            873.0
Comfort Septemberish    860.0
Margarito Nava          852.5
Eduardo Woodman         852.0
Rai Spliff              845.5
dtype: float64

## Ranking Pitchers

In [238]:
pitching_ranks = ranker(pitching_df, labels=["earned_run_average", "strikeouts", "whip", "hits_allowed"], asc_list=[True, True, True, False], top=10)
pitching_ranks

name
Agan Harrison        296.0
Agan Harrison        277.0
Edric Tosser         276.0
Edric Tosser         275.0
Peanut Bong          275.0
Percival Wheeler     274.5
Cory Twelve          264.5
Stephanie Winters    251.0
Sexton Wheerer       251.0
Mooney Doctor        249.0
dtype: float64

## Calculating better metrics
### Hitters:
#### Runs Created
- We will calculate Runs Created (RC)
The technical formula for this is 

    `((hits+walks-caught_stealing+hit_by_pitch-gidp) * (total_bases +(.26 * (walks-intentional_walks+hit_by_pitch)) + (.52 * ( sacrifice_hits+sacrifice_flies+stolen_bases))) / (at_bats+walks+hit_by_pitch+sacrifice_hit+sacrifice_flies)`
    
Note that we will be skipping the intentional walks (IBB) because this data is not tracked (or in the game, I guess, except for Charmed batters).

In [255]:
rc_df = hitting_df.assign(runs_created = lambda x: ((x.hits+x.walks-x.caught_stealing+x.hit_by_pitches)*
                          (x.total_bases + (.26 * (x.walks+x.hit_by_pitches) )+(.52*( x.sacrifice_bunts+x.sacrifice_flies+x.stolen_bases))))/
                         (x.at_bats+x.walks+x.hit_by_pitches+x.sacrifice_bunts+x.sacrifice_flies) )
q_widget = qgrid.show_grid(rc_df, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":50})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

# Put any other metrics in here and add them to the comparison
also add the ranker one

Let's compare a few different metrics:

In [306]:
rc_df[["name", "on_base_slugging"]].sort_values(by="on_base_slugging", ascending=False)

,name,on_base_slugging
133,Velasquez Meadows,2.0
69,Rhys Trombone,2.0
64,Miguel Javier,2.0
109,Lars Mendoza,2.0
1,Bennett Browning,2.0
...,...,...
70,Shaquille Torres,0.0
166,José Haley,0.0
163,Famous Oconnor,0.0
113,Paul Barnes,0.0


In [289]:
rank_compare = pd.DataFrame({"OBS": rc_df[["name", "on_base_slugging"]].sort_values(
                                        by="on_base_slugging", ascending=False)["name"].reset_index(
                                        drop=True), 
                             "OBS_drop": rc_df[rc_df["plate_appearances"] > 5][["name", "on_base_slugging"]].reindex(range(len(rc_df))).sort_values(
                                        by="on_base_slugging", ascending=False)["name"].reset_index(
                                        drop=True),
                            "RC": rc_df[["name", "runs_created"]].sort_values(
                                        by="runs_created",  ascending=False)["name"].reset_index(
                                        drop=True)})
q_widget = qgrid.show_grid(rank_compare, show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":150})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

OBS tends to be skewed in players with very few plate appearances, so we exclude those with less than 5 PA to get the `OBS_drop` column.

## Pitchers

Earned runs allowed is a decent metric, but does not tell the whole story, since it depends in part on the performance of the fielders. We will look at some Defense Independent Pitching Statistics (DIPS) to rectify this.

### Fielding Independent Pitching (FIP)
The formula is `(13*home_runs + 3*walks - 2*strikeouts)/innings_pitched) + C` where C is a constant term that recenters the average FIP to match the average ERA

### Expected Fielding Independent Pitching (xFIP)
This is the same as FIP except the home run rate is normalized to the league average (giving the "expected" rate of home runs).

The formula is `(( 13*(fly_balls*league_HR/FB%) ) + ( 3*(walks+hit_by_pitch)) - (2*strikeouts)) / innings_pitched) + C`

Okay, actually we can't calculate this because we don't have stats concerning fly balls.

In [297]:
print(pitching_df.columns)
print(hitting_df.columns)

Index(['name', 'id', 'team_abb', 'team_id', 'games', 'wins', 'losses',
       'win_pct', 'pitches_thrown', 'batters_faced', 'outs_recorded',
       'innings', 'runs_allowed', 'shutouts', 'quality_starts', 'strikeouts',
       'walks', 'home_runs_allowed', 'hits_allowed', 'hit_by_pitches',
       'earned_run_average', 'walks_per_9', 'hits_per_9', 'strikeouts_per_9',
       'home_runs_per_9', 'whip', 'strikeouts_per_walk'],
      dtype='object')
Index(['name', 'id', 'team_abb', 'team_id', 'batting_average',
       'on_base_percentage', 'slugging', 'appearances', 'plate_appearances',
       'at_bats', 'hits', 'walks', 'singles', 'doubles', 'triples',
       'quadruples', 'home_runs', 'runs_batted_in', 'strikeouts',
       'sacrifice_bunts', 'sacrifice_flies', 'at_bats_risp', 'hits_risp',
       'batting_average_risp', 'on_base_slugging', 'total_bases',
       'hit_by_pitches', 'ground_outs', 'flyouts', 'gidp', 'stolen_bases',
       'caught_stealing', 'runs'],
      dtype='object')


In [303]:
dips_df = pitching_df.assign(fip = lambda x: (((13*x.home_runs_allowed) + (3*x.walks) - (2*x.strikeouts))/x.innings))

dips_df["fip"] = dips_df["fip"] + (pitching_df["earned_run_average"].mean() - dips_df["fip"].mean())
q_widget = qgrid.show_grid(dips_df[["name", "earned_run_average", "fip", "whip", "strikeouts_per_9"]], show_toolbar=True, grid_options={"forceFitColumns":False, "defaultColumnWidth":150})
q_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [305]:
ranker(dips_df, ["earned_run_average", "fip", "whip", "strikeouts_per_9"], [False, False, False, True], top=10)

name
Wyatt Pothos          423.0
Burke Gonzales        421.0
Hiroto Wilcox         416.0
Theodore Cervantes    409.0
Gabriel Griffith      399.0
Dunlap Figueroa       399.0
Yosh Carpenter        391.5
Rafael Davids         390.0
Curry Aliciakeyes     388.0
Qais Dogwalker        385.5
dtype: float64